In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
raw_path = '/Users/davidcheung/Desktop/deep learning project -group2 additional work package/older versions/new data/'
data = pd.DataFrame()
data = data.append(pd.read_csv(raw_path+'1996-2001.csv', index_col=0))
data = data.append(pd.read_csv(raw_path+'2002-2007.csv', index_col=0))
data = data.append(pd.read_csv(raw_path+'2008-2013.csv', index_col=0))
data = data.append(pd.read_csv(raw_path+'2014-2017.csv', index_col=0))


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data.head()

,date,BIDLO,ASKHI,PRC,VOL,RET,BID,ASK,OPENPRC,NUMTRD,RETX
PERMNO,,,,,,,,,,,
10001,19960102,8.75,9.50,-9.125,0.0,-0.026667,8.75,9.5,NaN,0.0,-0.026667
10001,19960103,9.50,9.50,9.500,119.0,0.041096,8.75,9.5,9.50,1.0,0.041096
10001,19960104,8.75,9.50,-9.125,0.0,-0.039474,8.75,9.5,NaN,0.0,-0.039474
10001,19960105,8.75,8.75,8.750,1384.0,-0.041096,8.75,9.5,8.75,2.0,-0.041096
10001,19960108,8.75,9.50,8.750,1000.0,0.000000,8.75,9.5,9.50,2.0,0.000000


# Data summary
This block summarizes number of stocks, number of trading days, start and end dates, as well as features selected for the project

In [4]:
# Summary of data

stockList = np.unique(data.index.values)
tradingDates = np.unique(data.date)

numOfStock = len(stockList)
numOfTradingDate = len(tradingDates)
totalStart = tradingDates.min()
totalEnd = tradingDates.max()

print(numOfStock, 'stocks selected for the study')
print(numOfTradingDate, 'trading days selected for the study')
print('Starting from:', totalStart, 'ending from:', totalEnd)
print('Total features selected: ', data.columns.values[1:])
print('Total observation:', len(data))

19836 stocks selected for the study
5560 trading days selected for the study
Starting from: 19960102 ending from: 20171229
Total features selected:  ['BIDLO' 'ASKHI' 'PRC' 'VOL' 'RET' 'BID' 'ASK' 'OPENPRC' 'NUMTRD' 'RETX']
Total observation: 40531104


In [5]:
# Start with some reduced data: longest traded 500 stocks and two features(PRC and VOL)

_, counts = np.unique(data.index.values, return_counts=True)
top2000StocksList = stockList[np.argsort(counts)[-1500:]]
reduced_stock = data.loc[top2000StocksList][['date','PRC','VOL','BIDLO','ASKHI']]
reduced_stock.PRC = reduced_stock.PRC.abs()

In [6]:
reduced_stock.head()

,date,PRC,VOL,BIDLO,ASKHI
PERMNO,,,,,
63773,19960102,39.750,61400.0,39.125,39.875
63773,19960103,39.875,105000.0,39.500,40.125
63773,19960104,39.750,69500.0,39.500,40.000
63773,19960105,40.500,42800.0,39.375,40.500
63773,19960108,40.125,46300.0,40.125,40.625


In [7]:
# Eliminate stocks that have too many NaN in between

def hasConsecutiveNaN(series, numOfConsecutive=5):
    i=0
    for x in series:
        if np.isnan(x):
            flag=True
            i+=flag
            if i>=5:
                return True
        else:
            flag=False
    return False

tooManyNaN = []
for s in top2000StocksList:
    tooManyNaN.append(hasConsecutiveNaN(reduced_stock.loc[s].PRC))

In [8]:
reduced_stock = reduced_stock.loc[top2000StocksList[~np.array(tooManyNaN)]].fillna(method='ffill')

In [9]:
with open('reduced_stock','wb') as f:
    pickle.dump(reduced_stock, f)
    f.close()

In [10]:
reduced_stock.tail()

,date,PRC,VOL,BIDLO,ASKHI
PERMNO,,,,,
56274,20171222,38.08,2880973.0,37.92,38.26
56274,20171226,38.21,1890332.0,38.02,38.63
56274,20171227,38.07,2696672.0,37.95,38.36
56274,20171228,38.05,1876344.0,37.68,38.22
56274,20171229,37.67,2392097.0,37.65,38.24
